In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("mini_hw1") \
    .getOrCreate()

In [2]:
# Obtain departure Delays data
delay = spark.read.csv("s3n://csed516/Flights/flights.csv", header="true", inferSchema="true")
delay.registerTempTable("departureDelays")
delay.cache()
print(delay.columns)

# Obtain airports dataset
airports = spark.read.csv("s3n://csed516/Flights/airports.csv", header="true", inferSchema="true")
airports.registerTempTable("airports")
print(airports.columns)

['FL_DATE', 'CARRIER', 'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR', 'DEP_TIME', 'DEP_DELAY', 'ARR_DELAY']
['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE']


In [3]:
# create trip IATA codes table
tripIATA = sqlContext.sql("""
  SELECT DISTINCT IATA FROM(
    SELECT DISTINCT ORIGIN AS iata FROM departureDelays 
    UNION ALL SELECT DISTINCT DEST AS iata FROM departureDelays) x""")
tripIATA.registerTempTable("tripIATA")

# Merge airport data with tripIATA data
airports = sqlContext.sql("""
   SELECT IATA_CODE AS IATA, City, State, Country 
   FROM airports
   JOIN tripIATA trip ON trip.IATA = airports.IATA_CODE""")
airports.registerTempTable("airports")
airports.cache()

# Build `departureDelays_geo` DataFrame
# Obtain key attributes such as Date of flight, delays, distance,
# and airport information (Origin, Destination)
departureDelays_geo = sqlContext.sql("""
    SELECT CAST(f.FL_DATE as int) as tripid,
           CAST(CONCAT(CONCAT(CONCAT(CONCAT(CONCAT(CONCAT('2017-', CONCAT(CONCAT(SUBSTR(CAST(f.FL_DATE as string), 1, 2), '-')), SUBSTR(CAST(f.FL_DATE AS string), 3, 2)), ' '), SUBSTR(CAST(f.FL_DATE AS string), 5, 2)), ':'), SUBSTR(CAST(f.FL_DATE AS string), 7, 2)), ':00') as timestamp) as localdate,
           CAST(f.DEP_DELAY as int) as delay, 
           f.ORIGIN AS src, f.DEST AS dst, o.city AS city_src,
           d.city AS city_dst, 
           o.state AS state_src,
           d.state AS state_dst
    FROM departureDelays f 
    JOIN airports o 
        ON o.IATA = f.ORIGIN 
    JOIN airports d 
        ON d.IATA = f.DEST""")
departureDelays_geo.registerTempTable("departureDelays_geo")

# Cache and Count
departureDelays_geo.cache()
departureDelays_geo.count()

1526121

In [4]:
from pyspark.sql.functions import *
from graphframes import *

tripVertices = airports.withColumnRenamed("IATA", "id").distinct()
tripEdges = departureDelays_geo.select("tripid", "delay", "src", "dst", "city_dst", "state_dst")

# Cache the graph
tripEdges.cache()
tripVertices.cache()

# Examine the vertices and edges
tripVertices.show()
tripEdges.show()

# Build the graph
tripGraph = GraphFrame(tripVertices, tripEdges)
print tripGraph

# Project columns to build the smaller datastructure
tripEdgesPrime = departureDelays_geo.select("tripid", "delay", "src", "dst")
tripGraphPrime = GraphFrame(tripVertices, tripEdgesPrime)

+---+----------------+-----+-------+
| id|            City|State|Country|
+---+----------------+-----+-------+
|FAT|          Fresno|   CA|    USA|
|CMH|        Columbus|   OH|    USA|
|PHX|         Phoenix|   AZ|    USA|
|PAH|         Paducah|   KY|    USA|
|ITH|          Ithaca|   NY|    USA|
|COS|Colorado Springs|   CO|    USA|
|PSE|           Ponce|   PR|    USA|
|MYR|    Myrtle Beach|   SC|    USA|
|RNO|            Reno|   NV|    USA|
|VLD|        Valdosta|   GA|    USA|
|PSC|           Pasco|   WA|    USA|
|SRQ|        Sarasota|   FL|    USA|
|LAX|     Los Angeles|   CA|    USA|
|JMS|       Jamestown|   ND|    USA|
|CAE|        Columbia|   SC|    USA|
|DAY|          Dayton|   OH|    USA|
|MFR|         Medford|   OR|    USA|
|JFK|        New York|   NY|    USA|
|LAS|       Las Vegas|   NV|    USA|
|CLT|       Charlotte|   NC|    USA|
+---+----------------+-----+-------+
only showing top 20 rows

+----------+-----+---+---+----------+---------+
|    tripid|delay|src|dst|  city_dst|s

In [5]:
#number of vertices and edges?
print "Airports: %d" % tripGraph.vertices.count()
print "Trips: %d" % tripGraph.edges.count()

Airports: 299
Trips: 1526121


In [20]:
print tripGraphPrime
#number of vertices and edges?
print "Airports: %d" % tripGraphPrime.vertices.count()
print "Trips: %d" % tripGraphPrime.edges.count()

GraphFrame(v:[id: string, City: string ... 2 more fields], e:[src: string, dst: string ... 2 more fields])
Airports: 299
Trips: 1526121


In [36]:
tripGraph.degrees.sort(desc("degree")).head(1)

+---+------+
| id|degree|
+---+------+
|ATL|193125|
|ORD|153606|
|DEN|123894|
|LAX|115020|
|DFW| 96462|
|SFO| 94164|
|SEA| 80352|
|LAS| 79362|
|PHX| 79056|
|MSP| 75747|
|MCO| 71070|
|IAH| 69042|
|BOS| 66765|
|DTW| 65658|
|SLC| 60012|
|EWR| 59940|
|CLT| 58299|
|BWI| 55560|
|JFK| 50793|
|MDW| 48510|
+---+------+
only showing top 20 rows



[Row(id=u'ATL', degree=193125)]

In [14]:
#most number of flights
mostflights_in = tripGraph.edges.groupBy("src").count()
mostflights_in.show()

+---+-----+
|src|count|
+---+-----+
|BGM|  255|
|INL|  162|
|PSE|  279|
|DLG|   93|
|MSY|11859|
|PPG|   39|
|GEG| 3354|
|SNA|10818|
|BUR| 6462|
|GRB| 1227|
|GTF|  513|
|IDA|  669|
|GRR| 2976|
|JLN|  180|
|EUG| 1167|
|PSG|  186|
|PVD| 3420|
|MYR| 2166|
|GSO| 1548|
|OAK|13476|
+---+-----+
only showing top 20 rows



In [40]:
mostflights_out = tripGraph.edges.groupBy("dst").count()
mostflights_out.show()

+---+-----+
|dst|count|
+---+-----+
|BGM|  255|
|DLG|   93|
|INL|  162|
|PSE|  279|
|MSY|11862|
|PPG|   39|
|GEG| 3354|
|BUR| 6462|
|SNA|10818|
|GRB| 1230|
|GTF|  510|
|IDA|  669|
|GRR| 2976|
|JLN|  180|
|EUG| 1164|
|PSG|  186|
|GSO| 1548|
|MYR| 2166|
|PVD| 3420|
|OAK|13476|
+---+-----+
only showing top 20 rows



In [39]:
# Run PageRank algorithm, and show results.
results = tripGraph.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").sort("pagerank",ascending=False).take(30) ##get the top 30

[Row(id=u'ATL', pagerank=18.94035344317982),
 Row(id=u'ORD', pagerank=15.075045933905992),
 Row(id=u'DEN', pagerank=12.077673683793895),
 Row(id=u'LAX', pagerank=11.087205765693602),
 Row(id=u'DFW', pagerank=9.371725090750106),
 Row(id=u'SFO', pagerank=9.112308905290758),
 Row(id=u'SEA', pagerank=7.858366914119087),
 Row(id=u'PHX', pagerank=7.660053586576233),
 Row(id=u'LAS', pagerank=7.640562780154049),
 Row(id=u'MSP', pagerank=7.46256513039359),
 Row(id=u'MCO', pagerank=6.872546778864025),
 Row(id=u'IAH', pagerank=6.788605577015002),
 Row(id=u'DTW', pagerank=6.463561452881664),
 Row(id=u'BOS', pagerank=6.431703967882096),
 Row(id=u'SLC', pagerank=5.921566639479554),
 Row(id=u'EWR', pagerank=5.79108633089163),
 Row(id=u'CLT', pagerank=5.616949417169181),
 Row(id=u'BWI', pagerank=5.3591629165023615),
 Row(id=u'JFK', pagerank=4.9186114622316675),
 Row(id=u'MDW', pagerank=4.6730483476892335),
 Row(id=u'LGA', pagerank=4.555018999555448),
 Row(id=u'SAN', pagerank=4.548271420655757),
 Row(i